In [5]:
import numpy as np
import pandas as pd

# Load the Data

In [6]:
data = pd.read_csv('train.En.csv')
test = pd.read_csv('task_A_En_test.csv')

In [7]:
data = data[['tweet', 'sarcastic']]
data.rename(columns={'tweet': 'text'}, inplace=True)
data['text'] = data['text'].astype('string')
data['sarcastic'] = data['sarcastic'].astype('int')
data.dropna(inplace=True)
data

,text,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1
...,...,...
3463,The population spike in Chicago in 9 months is...,0
3464,You'd think in the second to last English clas...,0
3465,I’m finally surfacing after a holiday to Scotl...,0
3466,Couldn't be prouder today. Well done to every ...,0


# Tokenize Data

In [46]:
from sklearn.model_selection import train_test_split
from collections import Counter
import re

In [47]:
alpha = re.compile('[^a-z ]')

In [108]:
class Tokenizer:
  def __init__(self, sentences):
    words = [w for s in self.get_words(sentences) for w in s]
    self.wtoi = {w:i for i,(w,c) in enumerate(Counter(words).most_common())}
    self.n_words = len(self.wtoi)
  def get_words(self, sentences):
    return [alpha.sub('', s.lower()).split() for s in sentences]
  def tokenize(self, sentences):
    vec = np.zeros((len(sentences), self.n_words))
    for i, s in enumerate(self.get_words(sentences)):
      for w in s:
        if w in self.wtoi:
          vec[i][self.wtoi[w]] += 1
    return vec

tz = Tokenizer(data['text'])
x_train, x_valid, y_train, y_valid = train_test_split(tz.tokenize(data['text']), data['sarcastic'], test_size = 0.1)

# Train Model

In [122]:
from sklearn.tree import DecisionTreeClassifier

In [123]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
model.score(x_valid, y_valid)

0.6628242074927954

# Evaluate

In [124]:
from sklearn.metrics import f1_score
# Testing Randomness
f1_score(np.random.randint(0, 2, test['sarcastic'].shape), test['sarcastic'])

0.23314285714285718

## Testing our model

In [125]:
f1_score(model.predict(x_valid), y_valid)

0.17021276595744683

In [126]:
f1_score(model.predict(tz.tokenize(test['text'])), test['sarcastic'])

0.27722772277227725